In [6]:
import re

def extract_resume_sections(file_path):
    """
    Extract only LaTeX resume sections, skipping preamble and personal info.
    Returns a list of sections as strings.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Ensure document starts correctly
    if "\\begin{document}" not in content:
        raise ValueError("No \\begin{document} found in LaTeX file.")

    # print(content.split("\\begin{document}", 1)[0])
    # Extract everything after \begin{document}
    body_start = content.split("\\begin{document}", 1)[1]

    # Extract everything before \end{document}, if present
    body, sep, after = body_start.partition("\\end{document}")

    # Skip personal info: start from first \section or \section*
    first_section_match = re.search(r"\\section\*?|\\section", body)
    if not first_section_match:
        raise ValueError("No \\section found in LaTeX resume body.")

    sections_text = body[first_section_match.start():]

    # Split by section headers
    sections = re.split(r"(?=\\section\*?|\\section)", sections_text)
    sections = [sec.strip() for sec in sections if sec.strip()]

    return sections




if __name__ == "__main__":
    resume_path = '../Application_All/Resume/Resume_Ankit.tex'
    sections = extract_resume_sections(resume_path)
    
    # print("\nSections:")
    # for section in sections:
    #     print(section)
    

In [1]:
from Resume import ResumeModifier

In [10]:
sections = [sec for sec in sections]


In [2]:
import re

def extract_intro_and_sections(file_path):
    """
    Extract the personal info / intro and the sections separately from a LaTeX resume.
    Returns:
        intro: str (personal info at top)
        sections: list of strings (each LaTeX section)
    """
    with open(file_path, "r", encoding="utf-8") as f:
        content = f.read()

    # Everything between \begin{document} and \end{document}
    body_start = content.split("\\begin{document}", 1)[1]
    body, _, _ = body_start.partition("\\end{document}")

    # Split into intro (everything before first section) and sections
    match = re.search(r"\\section\*?|\\section", body)
    if not match:
        raise ValueError("No sections found in resume.")

    intro = body[:match.start()].strip()  # personal info
    sections_text = body[match.start():]

    # Split sections by \section or \section*
    sections = re.split(r"(?=\\section\*?|\\section)", sections_text)
    sections = [sec.strip() for sec in sections if sec.strip()]

    return intro, sections


In [3]:
def assemble_resume(intro, sections, original_file="resume.tex", output_file="tailored_resume.tex"):
    """
    Reassemble LaTeX resume keeping preamble, intro (personal info), and modified sections.
    """
    with open(original_file, "r", encoding="utf-8") as f:
        content = f.read()

    preamble, _, _ = content.partition("\\begin{document}")
    _, sep, end = content.rpartition("\\end{document}")
    end_document = sep + end if sep else "\\end{document}"

    # Join sections safely
    new_content = preamble + "\\begin{document}\n\n" + intro + "\n\n" + "\n\n".join(sections) + "\n\n" + end_document

    with open(output_file, "w", encoding="utf-8") as f:
        f.write(new_content)

    print(f"Tailored resume saved to {output_file}")


resume_path = '../Application_All/Resume/Resume_Ankit.tex'


# 1️⃣ Extract
intro, sections = extract_intro_and_sections(resume_path)

# # 2️⃣ Tailor sections using your model
# tailored_sections = tailor_sections_with_model(sections, job_desc)

# 3️⃣ Assemble final LaTeX resume
assemble_resume(intro, sections, original_file=resume_path, output_file="tailored_resume.tex")


Tailored resume saved to tailored_resume.tex


In [4]:
from Resume import ResumeModifier

resume = (ResumeModifier(
            resume_path='../Application_All/Resume/Resume_Ankit.tex',
            job_description='./job_description.txt',  # Add job description if available
            outdir='./test',  # Specify output directory if needed
            company_name='test'  # Specify company name if needed
            )
        .modify_resume()
        )

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerMinutePerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-1.5-pro"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
}
violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_input_token_count"
  quota_id: "GenerateContentInputTokensPerModelPerMinute-FreeTier"
  quota_dimensions 